In [3]:
import numpy as np
import pandas as pd
from pathlib import Path
from importlib import reload
import logging
from ast import literal_eval
from datetime import datetime

from utils import files, configs, logs
from utils.types import DbKey
from twitter_data import tweets
from dbschema import db

In [2]:
logs.setup_logger('transfer-to-pgsql',
                  desc='Moving all records to a pgsql database.')

In [3]:
log = logging.getLogger(__name__)

In [6]:
reload(db)

<module 'dbschema.db' from '/home/rimov/Documents/Code/NLP/lin-que-dropping/src/dbschema/db.py'>

In [4]:
path_e = files.choose_save_path('e')
path_c = files.choose_save_path('c')

Choose from the available folders, comma-separated (or "a" for all):
0. 2021-07-26-at-15:46:00
1. 2021-07-26-at-23:45:00
2. 2021-11-07-at-22:10:00
3. 2023-02-08-at-23:05:39
4. 2021-07-26-at-19:05:00
5. 2023-02-22-at-18:46:25
6. 2021-07-07-at-10:50:00
7. 2023-02-22-at-18:13:40
Return folder(s): 5
Choose from the available folders, comma-separated (or "a" for all):
0. 20210615-es-cleaned
1. 07112021-at-2210-combined
2. 2021-11-07-at-22:10:00
3. 20210726-combined
4. 20210726
5. 2023-02-22-at-18:46:25
6. 2021-07-07-at-10:50:00
7. 07112021-at-2210
8. 2021-07-11-at-22:10:00
Return folder(s): 5


In [5]:
path_e

[PosixPath('/home/rimov/Documents/Code/NLP/lin-que-dropping/data/extracted/saved/twitter/es/2023-02-22-at-18:46:25')]

In [6]:
path_e_tweets = path_e[0]/'tweets'/'es-twitter-decir-tweets-1-589.csv'
path_e_users = path_e[0]/'users'/'es-twitter-decir-users-1-1270.csv'
path_e_places = path_e[0]/'places'/'es-twitter-decir-places-1-470.csv'
path_e_meta = path_e[0]/'twitterdata'/'es-twitter-decir-twitterdata-2-3.csv'

paths_e_d = [path_e_tweets, path_e_users, path_e_places, path_e_meta]
print(list(map(lambda x: x.is_file(), paths_e_d)))

[True, True, True, True]


In [7]:
datas = list(map(lambda x: pd.read_csv(x, sep='~'), paths_e_d))

In [8]:
print(list(map(lambda x: x.info(), datas)))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 589 entries, 0 to 588
Data columns (total 16 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   referenced_tweets                398 non-null    object
 1   created_at                       589 non-null    object
 2   text                             589 non-null    object
 3   edit_history_tweet_ids           589 non-null    object
 4   lang                             589 non-null    object
 5   id                               589 non-null    int64 
 6   author_id                        589 non-null    int64 
 7   public_metrics.retweet_count     589 non-null    int64 
 8   public_metrics.reply_count       589 non-null    int64 
 9   public_metrics.like_count        589 non-null    int64 
 10  public_metrics.quote_count       589 non-null    int64 
 11  public_metrics.impression_count  589 non-null    int64 
 12  entities.mentions                392

In [9]:
print(list(map(lambda x: display(x.head(3)), datas)))

,referenced_tweets,created_at,text,edit_history_tweet_ids,lang,id,author_id,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,public_metrics.quote_count,public_metrics.impression_count,entities.mentions,geo.place_id,geo.coordinates.type,geo.coordinates.coordinates
0,"[{'type': 'replied_to', 'id': '162854163504166...",2023-02-22T23:46:49.000Z,@palitoborrotw @xmilianx_ Cuando dijo eso palito,['1628541852012904455'],es,1628541852012904455,880749655935660032,0,0,0,0,0,"[{'start': 0, 'end': 14, 'username': 'palitobo...",000a15027a9003e0,NaN,NaN
1,"[{'type': 'replied_to', 'id': '162852917090816...",2023-02-22T23:46:47.000Z,@AlfredoDarrigo @JMilei Dicen poder aplicar ma...,['1628541846707224576'],es,1628541846707224576,2799221573,0,0,0,0,0,"[{'start': 0, 'end': 15, 'username': 'AlfredoD...",00f84d414936f28e,NaN,NaN
2,NaN,2023-02-22T23:46:36.000Z,¿Vos tenías ganas de enamorarte?\nPorque despu...,['1628541799462469633'],es,1628541799462469633,229175968,0,0,0,0,1,NaN,6417871953fa5e86,NaN,NaN


,location,username,id,name,created_at,public_metrics.followers_count,public_metrics.following_count,public_metrics.tweet_count,public_metrics.listed_count
0,"Cañada de Gomez, Argentina",LucioPasquali,880749655935660032,Lucio Pasquali,2017-06-30T11:27:36.000Z,647,423,25092,0
1,Argentina,palitoborrotw,1546666012187054080,palito borro un tweet ⭐️⭐️⭐️,2022-07-12T01:21:40.000Z,4803,38,3211,0
2,Villa Luro,xmilianx_,1242101574001967105,Palito,2020-03-23T14:51:16.000Z,119554,1180,59372,85


,country,full_name,id
0,Argentina,"Cañada de Gomez, Argentina",000a15027a9003e0
1,Argentina,"Córdoba, Argentina",00f84d414936f28e
2,United States,"Silver Spring, MD",6417871953fa5e86


,parameter,resource_id,value,detail,title,resource_type,type
0,entities.mentions.username,follambula,follambula,User has been suspended: [follambula].,Forbidden,user,https://api.twitter.com/2/problems/resource-no...
1,entities.mentions.username,AlexaCindy,AlexaCindy,User has been suspended: [AlexaCindy].,Forbidden,user,https://api.twitter.com/2/problems/resource-no...
2,entities.mentions.username,jmijarra,jmijarra,User has been suspended: [jmijarra].,Forbidden,user,https://api.twitter.com/2/problems/resource-no...


[None, None, None, None]


In [10]:
verbs = set(files.get_verb_conjugations()['verb'].unique())
print(verbs)

{'prometer', 'conseguir', 'suponer', 'recomendar', 'parecer', 'negar', 'comprobar', 'sentir', 'desear', 'mandar', 'gritar', 'dudar', 'demostrar', 'reclamar', 'solicitar', 'lamentar', 'saber', 'temer', 'confesar', 'pensar', 'ver', 'considerar', 'mencionar', 'responder', 'ojala', 'creer', 'confirmar', 'lograr', 'pedir', 'adivinar', 'apostar', 'suplicar', 'contar', 'entender', 'esperar', 'afirmar', 'predecir', 'querer', 'mostrar', 'admitir', 'jurar', 'imaginar', 'ordenar', 'recordar', 'prever', 'acordar', 'rogar', 'decir', 'suspirar', 'asegurar'}


In [44]:
ts = datas[0]
ts.head()

,referenced_tweets,created_at,text,edit_history_tweet_ids,lang,id,author_id,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,public_metrics.quote_count,public_metrics.impression_count,entities.mentions,geo.place_id,geo.coordinates.type,geo.coordinates.coordinates
0,"[{'type': 'replied_to', 'id': '162854163504166...",2023-02-22T23:46:49.000Z,@palitoborrotw @xmilianx_ Cuando dijo eso palito,['1628541852012904455'],es,1628541852012904455,880749655935660032,0,0,0,0,0,"[{'start': 0, 'end': 14, 'username': 'palitobo...",000a15027a9003e0,NaN,NaN
1,"[{'type': 'replied_to', 'id': '162852917090816...",2023-02-22T23:46:47.000Z,@AlfredoDarrigo @JMilei Dicen poder aplicar ma...,['1628541846707224576'],es,1628541846707224576,2799221573,0,0,0,0,0,"[{'start': 0, 'end': 15, 'username': 'AlfredoD...",00f84d414936f28e,NaN,NaN
2,NaN,2023-02-22T23:46:36.000Z,¿Vos tenías ganas de enamorarte?\nPorque despu...,['1628541799462469633'],es,1628541799462469633,229175968,0,0,0,0,1,NaN,6417871953fa5e86,NaN,NaN
3,"[{'type': 'replied_to', 'id': '162841558618208...",2023-02-22T23:46:31.000Z,@MaihenH @PoleoRafael Para nadie es un secreto...,['1628541777882800130'],es,1628541777882800130,101557817,0,0,0,0,0,"[{'start': 0, 'end': 8, 'username': 'MaihenH',...",015d19216c749864,NaN,NaN
4,"[{'type': 'replied_to', 'id': '162854175502391...",2023-02-22T23:46:27.000Z,😏 Tiene que estar arriba 🆙 \n\nSi es un serum ...,['1628541760275087361'],es,1628541760275087361,1398853015042469889,0,1,0,0,2,NaN,0177894212b08f73,NaN,NaN


In [45]:
ts['referenced_tweets'].iloc[2] is np.nan

True

In [46]:
refd = ts['referenced_tweets'].apply(lambda x: literal_eval(x) if x is not np.nan else None)
refd.head()

0    [{'type': 'replied_to', 'id': '162854163504166...
1    [{'type': 'replied_to', 'id': '162852917090816...
2                                                 None
3    [{'type': 'replied_to', 'id': '162841558618208...
4    [{'type': 'replied_to', 'id': '162854175502391...
Name: referenced_tweets, dtype: object

In [32]:
refd.apply(lambda x: len(x) if x is not None else 0).sort_values()

176    0
179    0
473    0
472    0
470    0
      ..
234    1
230    1
228    1
247    1
588    1
Name: referenced_tweets, Length: 589, dtype: int64

In [35]:
for t in refd.iloc[588]:
    print(t['id'])

1628511098742661122


In [47]:
ts.iloc[588]['created_at']

'2023-02-22T21:49:42.000Z'

In [49]:
datetime.strptime(ts.iloc[588]['created_at'], '%Y-%m-%dT%H:%M:%S.%fZ')

datetime.datetime(2023, 2, 22, 21, 49, 42)

In [50]:
mens = ts['entities.mentions']
mens.head()

0    [{'start': 0, 'end': 14, 'username': 'palitobo...
1    [{'start': 0, 'end': 15, 'username': 'AlfredoD...
2                                                  NaN
3    [{'start': 0, 'end': 8, 'username': 'MaihenH',...
4                                                  NaN
Name: entities.mentions, dtype: object

In [51]:
mens.iloc[3]

"[{'start': 0, 'end': 8, 'username': 'MaihenH', 'id': '1610496259'}, {'start': 9, 'end': 21, 'username': 'PoleoRafael', 'id': '307549658'}]"

In [52]:
edits = ts['edit_history_tweet_ids']
edits.head()

0    ['1628541852012904455']
1    ['1628541846707224576']
2    ['1628541799462469633']
3    ['1628541777882800130']
4    ['1628541760275087361']
Name: edit_history_tweet_ids, dtype: object

In [54]:
edits.apply(literal_eval).apply(len).sort_values()

0      1
388    1
389    1
390    1
391    1
      ..
197    1
198    1
199    1
183    1
588    1
Name: edit_history_tweet_ids, Length: 589, dtype: int64

In [55]:
us = datas[1]
us.head()

,location,username,id,name,created_at,public_metrics.followers_count,public_metrics.following_count,public_metrics.tweet_count,public_metrics.listed_count
0,"Cañada de Gomez, Argentina",LucioPasquali,880749655935660032,Lucio Pasquali,2017-06-30T11:27:36.000Z,647,423,25092,0
1,Argentina,palitoborrotw,1546666012187054080,palito borro un tweet ⭐️⭐️⭐️,2022-07-12T01:21:40.000Z,4803,38,3211,0
2,Villa Luro,xmilianx_,1242101574001967105,Palito,2020-03-23T14:51:16.000Z,119554,1180,59372,85
3,"Córdoba, Argentina",franco280202,2799221573,Soy Franco,2014-10-02T01:33:23.000Z,1151,2109,50516,2
4,NaN,AlfredoDarrigo,4234892896,Alfredo D'Arrigo,2015-11-20T13:42:25.000Z,56948,50621,155809,63


In [56]:
us['id'].unique()

array([ 880749655935660032, 1546666012187054080, 1242101574001967105, ...,
       1551439316206792704,           234810756,           868207644])